# MIDI to Audio to Ardour Pipeline Demo

This notebook demonstrates the complete workflow:
1. Generate MIDI files programmatically
2. Render MIDI to audio using FluidSynth
3. Prepare for import to Ardour via ardour-mcp

## Requirements

```bash
# Python packages (already installed in ardour-mcp env)
uv pip install midiutil pretty_midi

# System packages (for audio rendering)
sudo apt-get install fluidsynth fluid-soundfont-gm
```

In [ ]:
# Import required libraries
import sys
from pathlib import Path

# Add scripts directory to path
sys.path.insert(0, str(Path.cwd().parent / 'scripts'))

from midi_to_audio_pipeline import MIDIGenerator, FluidSynthRenderer
from IPython.display import Audio, display
import subprocess

## 1. Generate MIDI Files

We'll create three types of MIDI patterns:
- 808 Bass Pattern
- Drum Beat
- Melodic Sequence

In [ ]:
# Initialize MIDI generator
output_dir = Path("/tmp/midi_test")
generator = MIDIGenerator(output_dir=str(output_dir))

# Generate MIDI files
print("Generating MIDI files...\n")

bass_midi = generator.create_808_bass_pattern("808_bass.mid")
drums_midi = generator.create_drum_pattern("drum_beat.mid")
melody_midi = generator.create_melodic_sequence("melody.mid")

midi_files = [bass_midi, drums_midi, melody_midi]

print(f"\nGenerated {len(midi_files)} MIDI files:")
for f in midi_files:
    size_kb = f.stat().st_size / 1024
    print(f"  - {f.name} ({size_kb:.1f} KB)")

## 2. Inspect MIDI Files

Let's examine the MIDI files we created.

In [ ]:
# Use 'file' command to check MIDI format
for midi_file in midi_files:
    result = subprocess.run(
        ["file", str(midi_file)],
        capture_output=True,
        text=True
    )
    print(result.stdout.strip())

## 3. Render MIDI to Audio (Optional)

If FluidSynth is installed, render the MIDI files to WAV audio.

**Note**: This cell will fail gracefully if FluidSynth is not installed.

In [ ]:
# Check if FluidSynth is available
try:
    result = subprocess.run(
        ["fluidsynth", "--version"],
        capture_output=True,
        text=True,
        timeout=5
    )
    fluidsynth_available = (result.returncode == 0)
    if fluidsynth_available:
        print("FluidSynth found:")
        print(result.stdout.strip())
    else:
        print("FluidSynth not available")
except (FileNotFoundError, subprocess.TimeoutExpired):
    fluidsynth_available = False
    print("FluidSynth not installed")
    print("\nTo install: sudo apt-get install fluidsynth fluid-soundfont-gm")

In [ ]:
# Render MIDI to audio if FluidSynth is available
audio_files = []

if fluidsynth_available:
    try:
        print("Rendering MIDI files to audio...\n")
        renderer = FluidSynthRenderer()
        
        for midi_file in midi_files:
            try:
                audio_file = renderer.render(midi_file)
                audio_files.append(audio_file)
                print(f"✓ Rendered: {audio_file.name}")
            except Exception as e:
                print(f"✗ Failed to render {midi_file.name}: {e}")
        
        print(f"\nRendered {len(audio_files)} audio files")
        
    except Exception as e:
        print(f"Rendering error: {e}")
        print("\nMake sure FluidSynth and soundfonts are properly installed:")
        print("  sudo apt-get install fluidsynth fluid-soundfont-gm")
else:
    print("Skipping audio rendering (FluidSynth not available)")
    print("\nMIDI files are still available for manual rendering or import.")

## 4. Preview Audio (If Rendered)

Listen to the rendered audio files in the notebook.

In [ ]:
# Display audio players for rendered files
if audio_files:
    for audio_file in audio_files:
        if audio_file.exists():
            print(f"\n{audio_file.stem.replace('_', ' ').title()}:")
            display(Audio(str(audio_file)))
else:
    print("No audio files to preview.")
    print("\nGenerate audio with:")
    print("  uv run python scripts/midi_to_audio_pipeline.py")

## 5. Prepare for Ardour Import

Now we'll document the steps to import these files into Ardour using ardour-mcp.

In [ ]:
# Generate import instructions
print("=" * 70)
print("Ardour Import Workflow")
print("=" * 70)

if audio_files:
    print("\nAudio files ready for import:")
    for audio_file in audio_files:
        print(f"  - {audio_file}")
else:
    print("\nMIDI files ready (render to audio first):")
    for midi_file in midi_files:
        print(f"  - {midi_file}")

print("\n\nSteps to import into Ardour:")
print("-" * 70)

print("""
1. Start Ardour and create/open a session

2. Enable OSC in Ardour:
   - Edit → Preferences → Control Surfaces
   - Enable "Open Sound Control (OSC)"
   - Set port to 3819, enable all feedback

3. Use Claude with ardour-mcp to create tracks:
   
   Prompt: "Create three stereo audio tracks called:
            - 808 Bass
            - Drum Beat  
            - Melody
            
            Set them all to 0dB gain and enable input monitoring."

4. Import audio files manually in Ardour:
   - Session → Import
   - Select the WAV files from /tmp/midi_test/
   - Choose "Add files as new tracks" or drag to existing tracks

5. Use Claude to set up mixing:
   
   Prompt: "Set up the mix:
            - Pan bass to center
            - Pan drums 20% left
            - Pan melody 20% right
            - Create a Mix Bus
            - Route all three tracks to the Mix Bus at -6dB
            - Check for clipping and phase issues"

6. Continue mixing with Claude:
   - Add effects/plugins
   - Set up automation
   - Monitor levels and loudness
   - Export final mix
""")

print("=" * 70)

## 6. Example MCP Commands

These are the MCP tools that ardour-mcp provides for controlling Ardour.

In [ ]:
# Display example MCP commands as JSON
import json

example_commands = [
    {
        "tool": "create_audio_track",
        "arguments": {
            "name": "808 Bass",
            "channels": 2
        },
        "description": "Create stereo track for bass"
    },
    {
        "tool": "set_track_gain",
        "arguments": {
            "track_id": 1,
            "gain_db": 0.0
        },
        "description": "Set track 1 to unity gain"
    },
    {
        "tool": "set_track_pan",
        "arguments": {
            "track_id": 1,
            "pan_position": 0.0
        },
        "description": "Pan track 1 to center"
    },
    {
        "tool": "create_send",
        "arguments": {
            "track_id": 1,
            "bus_name": "Reverb Bus",
            "send_db": -12.0
        },
        "description": "Add reverb send at -12dB"
    },
    {
        "tool": "get_track_peak_level",
        "arguments": {
            "track_id": 1
        },
        "description": "Check peak level on track 1"
    }
]

print("Example MCP Commands for Ardour Control:\n")
for i, cmd in enumerate(example_commands, 1):
    print(f"{i}. {cmd['description']}")
    print(f"   Tool: {cmd['tool']}")
    print(f"   Arguments: {json.dumps(cmd['arguments'], indent=6)}")
    print()

## 7. Summary

This notebook demonstrated:

1. **MIDI Generation**: Created 808 bass, drums, and melody using Python
2. **Audio Rendering**: Converted MIDI to WAV using FluidSynth (if available)
3. **Ardour Integration**: Prepared files for import via ardour-mcp

### Complete Workflow

```
Python (midiutil) → MIDI files
       ↓
FluidSynth → WAV audio
       ↓
Ardour (manual import) → Tracks with audio
       ↓
Claude + ardour-mcp → Professional mixing
```

### Next Steps

- Install FluidSynth if not already: `sudo apt-get install fluidsynth fluid-soundfont-gm`
- Run full pipeline: `uv run python scripts/midi_to_audio_pipeline.py`
- Start Ardour and enable OSC
- Use Claude with ardour-mcp for mixing

### Resources

- [MIDI to Audio Pipeline Docs](../docs/MIDI_TO_AUDIO_PIPELINE.md)
- [ardour-mcp README](../README.md)
- [FluidSynth Documentation](https://www.fluidsynth.org/)
